<a href="https://colab.research.google.com/github/DasagociCorpus/Pruebas/blob/main/TP1/Profesional_III_(2400015V)_Prueba_Nombres_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **a. Supuestos**

**Exactitud en las Etiquetas de "Tipo de Entidad":** Las etiquetas de 'PARTY_TYPE_DESC' son precisas y correctamente categorizadas como 'ORGANIZACION' y 'INDIVIDUAL'.

**Distribución de los Datos:** La distribución de los nombres de las entidades (PARTY_NAME) en la muestra, es suficientemente variada lo cual permite un entrenamiento de un modelo robusto.

**Representatividad en la muestra:** La muestra de datos cargada desde Google Sheets es representativa y no contiene sesgos significativos.

**Precisión del Conteo de Tokens:** La transformación de texto a vectores de recuento por CountVectorizer captura adecuadamente la información relevante de los nombres de las entidades permitiendo la caracterización eficiente de los nombres/razones sociales.

# **b. Marco Teórico**

**Procesamiento de Texto:** CountVectorizer se utiliza para transformar un texto dado en un vector en función de la frecuencia (recuento) de cada palabra que aparece en todo el texto. Esto es útil cuando tenemos múltiples textos de este tipo y deseamos convertir cada palabra de cada texto en vectores (para usar en análisis de texto posteriores). (https://barcelonageeks.com/uso-de-countvectorizer-para-extraer-caracteristicas-de-texto/)

En términos técnicos,  se utiliza para convertir una colección de documentos de texto en una matriz de recuentos de tokens, lo cual permite que los modelos de aprendizaje automático procesen texto.


**Modelos de Clasificación:** Dentro del estado del arte para modelos de clasificación, existen diferentes referencias en cuanto a uso, eficiencia, capacidad, etc. De hecho, "La clasificación supervisada es una de las tares que más frecuentemente son llevadas a cabo por los denominados Sistemas Inteligentes. Por lo tanto, un gran número de paradigmas desarrollados bien por la Estadística (Regresión Logística, Análisis Discriminante) o bien por la Inteligencia Artificial (Redes Neuronales, Inducción de Reglas, Árboles de Decisión, Redes Bayesianas) son capaces de realizar las tareas propias de la clasificación." (https://bookdown.org/content/2274/metodos-de-clasificacion.html)

En este caso práctico, los bosques aleatorios (Random Forest) es un metaestimador que ajusta una serie de clasificadores de árboles de decisión en varias submuestras del conjunto de datos y utiliza promedios para mejorar la precisión predictiva y controlar el sobreajuste. (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier)

**Validación Cruzada:** Es una técnica utilizada para evaluar la eficacia de un modelo al dividir los datos en múltiples subconjuntos y entrenar el modelo en algunos de ellos mientras se evalúa en los restantes. (https://cienciadedatos.net/documentos/30_cross-validation_oneleaveout_bootstrap) Es de esta manera que logra proporcionar una mejor estimación del rendimiento del modelo en datos no vistos.

# **c. Exploración y Tratamiento de Datos**

## **Importar librerías**

Descripción de las librerías a usar:

- **pandas** para manipulación de datos.
- **train_test_split** y **cross_val_score** de sklearn para dividir los datos y validar el modelo.
- **CountVectorizer** para convertir texto en características numéricas.
- **RandomForestClassifier** para el modelo de clasificación.
- **classification_report** y **confusion_matrix** para evaluar el modelo.
- **seaborn** y **matplotlib** para visualización.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


## **Cargar y preparar los datos**

Primero, generamos la carga de los datos desde un archivo CSV en un Google Sheets. Segundo, eliminamos las filas duplicadas. Tercero, redefinimos las etiquetas de 'PARTY_TYPE_DESC' a valores numéricos: 'ORGANIZACION' a 0 y 'INDIVIDUAL' a 1 para implementación posterior del modelo.

In [ ]:
link = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSNIT8z9zZL7TGSBzAczlwsA5FT0q3_oUQFJfoxiLWa4SmtJb56mEP_66HzMG_QZLFj5wIgsBesYsTB/pub?gid=1098929358&single=true&output=csv'
df = pd.read_csv(link).drop_duplicates()

## **Dividir datos en entrenamiento y prueba**

En primera instancia, se hace uso del método CountVectorizer el cual:

> Convierte una colección de documentos de texto en una matriz de recuentos de tokens. Esta implementación produce una representación escasa de los recuentos utilizando scipy.sparse.csr_matrix. ]En el caso de no proporcionar] un diccionario a priori, [...] entonces la cantidad de características será igual al tamaño del vocabulario encontrado al analizar los datos. (https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)


Para lograr un modelamiento con resultados significativos,
'X' se define como la columna de nombres de las entidades (PARTY_NAME).
'y' se define como las etiquetas de tipo de entidad (PARTY_TYPE_DESC).

Las proporciones entre el conjunto de datos de entrenamiento y prueba son 70% y 30%.


In [ ]:
# División de características y etiquetas
X = CountVectorizer().fit_transform(df['PARTY_NAME'])
y = df['PARTY_TYPE_DESC']


In [ ]:
# División del df en datos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

El modelo de clasificación implementado es el de Bosques aleatorios el cual:

> es un metaestimador que ajusta una serie de clasificadores de árboles de decisión en varias submuestras del conjunto de datos y utiliza promedios para mejorar la precisión predictiva y controlar el sobreajuste. (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

Posterior a ello, realizamos predicciones sobre los datos de prueba (X_test).
Usamos cross_val_score para evaluar el modelo mediante validación cruzada con 5 pliegues (cv=5) y por último calculamos la matriz de confusión para comparar las predicciones con las etiquetas reales.



In [ ]:
model = RandomForestClassifier().fit(X_train, y_train)

y_pred = model.predict(X_test)
scores = cross_val_score(model, X, y, cv=5)
cm = confusion_matrix(y_test, y_pred)

## **Visualización y Reporte de clasificación y precisión de validación cruzada**

En esta instancia, se crea un gráfico de la matriz de confusión usando seaborn, este estará compuesto por xticklabels y yticklabels etiquetan los ejes con las clases 'ORGANIZACION' e 'INDIVIDUAL'.

Concluimos el proceso, imprimiendo el informe de clasificación que muestra la precisión, recall, F1-score y soporte para cada clase y el reporte de la precisión media de la validación cruzada y su desviación estándar.

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='g', xticklabels=['ORGANIZACION', 'INDIVIDUAL'], yticklabels=['ORGANIZACION', 'INDIVIDUAL'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Matriz de Confusión')
plt.show()

In [ ]:
print(classification_report(y_test, y_pred))
print(f"Precisión de la validación cruzada: {scores.mean():.2f} (+/- {scores.std() * 2:.2f})")

# **d. Modelos o Metodologías Utilizadas**

CountVectorizer, Random Forest Classifier y Validación Cruzada

# **e. Elección del Mejor Modelo/Metodología y los Criterios de su Selección**

Los criterios de Selección para el modelo y/o metodología fueron ptincipalmente las generadas por la matriz de confusión donde se recalca:
* **Precisión:** La capacidad del modelo para predecir correctamente las etiquetas.
* **Recall:** La capacidad del modelo para capturar todos los casos positivos.
* **F1-score:** El equilibrio entre precisión y recall.

Adicionalmente, contribuye en la elección la consistencia evaluada a través de la validación cruzada, la cual nos halba de una proporción general de predicciones correctas.


# **f. Conclusiones y Recomendaciones para el Trabajo Futuro**

**Conclusiones**

De acuerdo a los resultados generados por el modelo en la Matriz de confusión podemos validar cinco factores imporantes, a saber:

**Precision:** La precisión indica qué tan precisas son las predicciones del modelo cuando predice una clase específica. Para la aplicación actual, determinar el tipo de persona natural (Individual) o juridica (Organización) de acuerdo a la Razón social o nombre, la precisión para la clase '0' (organización) indique que el 93% de las veces que el modelo predice que un nombre pertenece a una organización, realmente lo es. Para la clase '1' (individual), indica que el 97% de las veces que el modelo predice que un nombre pertenece a un individuo, realmente lo es.

**Recall:** también conocido como "sensibilidad", indica qué tan bien el modelo captura todos los casos positivos. En este caso, el recall para la clase '0' (organización) nos dice que identifica correctamente el 98% de todas las organizaciones en el conjunto de datos. Para la clase '1' (individual), el 91% de todos los individuos en el conjunto de datos.

**F1-score:** es la media armónica de precision y recall. Es una medida útil para resumir el rendimiento de un clasificador en una sola métrica. Un F1-score cercano a 1 indica un buen equilibrio entre precision y recall. En este caso, los F1-scores son del 95% y 94% para las clases '0' y '1' respectivamente, lo que sugiere un buen equilibrio entre precision y recall en ambos casos.

**Accuracy:** La exactitud indica la proporción de predicciones correctas realizadas por el modelo en general. Para el presnte modelo, la exactitud es del 95%, lo que significa que el 95% de todas las predicciones del modelo son correctas.

**Validación Cruzada:**
Adicionalmente, sumamos los resultados generados desde la Validación Cruzada el cual fue de un 95% con una desviación estándar de +/- 0.02. Esto indica que el modelo generaliza bien a datos no vistos y tiene una consistencia razonable en su rendimiento.


---
**Recomendaciones para el Trabajo Futuro:**

**Optimización de Hiperparámetros:** Explorar y ajustar los hiperparámetros del Random Forest para potencialmente mejorar aún más el rendimiento del modelo.

**Robustecer el proceso de recolecció de datos:** Incluir datos y características tipificados, tabulados podría ayudar a mejorar la precisión del modelo y su capacidad para generalizar.

**Validación de Otros Modelos:** Validar otros modelos de clasificación tales como Support Vector Machines (SVM) o Gradient Boosting que, aunque sacrifican rendimiento por precisión, podrían suponer un debate sobre cuál modelo usar y sus posibles mejoras según costo-beneficio.

**Análisis de Errores:** Realizar un análisis detallado de los errores de clasificación para identificar patrones o características específicas que puedan estar causando problemas y ajustar el modelo en consecuencia.

**Actualización Continua del Modelo:** Implementar un proceso de actualización continua del modelo para incorporar nuevos datos y adaptar el modelo a posibles cambios en los patrones de los datos con el tiempo.

**Automatización del Pipeline:** Desarrollar un pipeline automatizado que abarque desde la carga y preprocesamiento de datos hasta la evaluación y actualización del modelo, facilitando el mantenimiento y mejora continua del sistema.

**Validación en Tiempo Real:** Implementar un sistema para validar el modelo en tiempo real, que permita un aprendizaje semisupervisado.



## **Información de Sistema**

In [ ]:
!pip install sinfo
from sinfo import sinfo
sinfo()